### Links zu den Daten
[Strecke Link](https://geovdbn.deutschebahn.com/pgv-map/geoserver.action?LAYERS=ISR%3AISR_V_GEO_TEN_KLASSIFIZIERUNG&TRANSPARENT=TRUE&FORMAT=kml&VERSION=1.1.1&TILED=false&USERDEFINEDSLD=&SERVICE=WMS&REQUEST=GetMap&VIEWPARAMS=ZEITSCHEIBE%3AUNDEFINED%3BLANG%3ADE%3BALG_DBNETZ_STRECKE%3Aalle%20Strecken%3BJAHR%3A2020&SRS=EPSG%3A31467&BBOX=3250000,5200000,3950000,6100000&WIDTH=700&HEIGHT=900)


[Bahnhöfe Link](https://geovdbn.deutschebahn.com/pgv-map/geoserver.action?LAYERS=ISR%3AISR_V_GEO_BETRIEBSSTELLEN_PUNKT&TRANSPARENT=TRUE&FORMAT=kml&VERSION=1.1.1&TILED=false&USERDEFINEDSLD=&SERVICE=WMS&REQUEST=GetMap&VIEWPARAMS=ZEITSCHEIBE%3AUNDEFINED%3BLANG%3ADE%3BALG_DBNETZ_STRECKE%3Aalle%20Strecken%3BJAHR%3A2020&SRS=EPSG%3A31467&BBOX=3250000,5200000,3950000,6100000&WIDTH=700&HEIGHT=900)

In [1]:
import fiona
import geopandas as gpd
from geopandas.plotting import plot_linestring_collection
import matplotlib.pyplot as plt
import shapely
from shapely.geometry import GeometryCollection
import networkx as nx
import random
import lxml.etree as etree
import re
import pandas as pd
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

# enable KML support
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

parser = etree.XMLParser(encoding='utf-8', collect_ids=False)

In [ ]:
def xml_to_json(xml):
    """a recursive function to convert xml to list dict mix

    Arguments:
        xml {etree} -- the xml to convert

    Returns:
        dict -- a dict list mix of the xml
    """
    parsed = dict(xml.attrib)
    for xml_child in list(xml):
        if xml_child.tag in parsed:
            parsed[xml_child.tag].append(xml_to_json(xml_child))
        else:
            parsed[xml_child.tag] = [xml_to_json(xml_child)]
    return parsed

### Read kml files

In [ ]:
strecke = gpd.read_file('../data/ISR-ISR_V_GEO_TEN_KLASSIFIZIERUNG.kml')
# bahnhöfe = gpd.read_file('../data/ISR-ISR_V_GEO_BETRIEBSSTELLEN_PUNKT.kml')

transform 
```python
    GeometryCollection[
        GeometryCollection[
            ...
        ]
    ]
```
to 
```python
    GeometryCollection[
        ...
    ]
```

In [ ]:
for i in range(len(strecke)):
    new_geo =  []
    for g in strecke.iat[i, 2]:
        if type(g) == shapely.geometry.collection.GeometryCollection:
            for u in g:
                new_geo.append(u)
        elif type(g) != shapely.geometry.point.Point:
            new_geo.append(g)
    strecke.iat[i, 2] = GeometryCollection(new_geo)

In [ ]:
atr_name = re.compile(r"<span class=\"atr-name\">(.*?)<\/span>")
atr_value = re.compile(r'<span class=\"atr-value\">(.*?)<\/span>')

strecke_atr = []
with tqdm(total=len(strecke)) as pbar:
    for i in range(len(strecke)):
        attributes = strecke.iat[i, 1]
        names = [match[1] for match in atr_name.finditer(attributes)]
        values = [match[1] for match in atr_value.finditer(attributes)]
        strecke_atr.append(pd.Series(values, names))
        pbar.update(1)

strecke_atr = pd.concat(strecke_atr, axis=1).T

In [ ]:
strecke_atr

In [ ]:
number_of_colors = len(strecke)

color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]
strecke['colors'] = color

In [ ]:
ax = strecke.plot(column='colors', figsize=(21*2, 24*2))
bahnhöfe.plot(ax=ax)

In [ ]:
bahnhöfe.plot(figsize=(21*2, 24*2))